<a href="https://colab.research.google.com/github/KwagEunsang/AI/blob/main/(LAIDD)_Cancer_genome_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2강. Cancer Mutation 1

In [ ]:
# Set working directory
setwd()  # command in terminal
list.files()  # check if the current directory is a correct one

# Read 'essential part' of maf
tcga.mut = read.csv("mc3.v0.2.8.PUBLIC.maf", header=T, sep="\t", colClasses =
  c("character", "NULL", "NULL", "NULL", "character", "integer", "integer", "NULL",
    "character", "character", "character", "NULL", "character", "NULL", "NULL",
    "character", "character", rep("NULL", 17), "character", "NULL", "character",
    "NULL", "NULL", rep("integer", 6), rep("NULL", 69)))

# check the number of entries
dim(tcga.mut)

# first 10 lines of the data (also use tail(tcga.mut))
head(tcga.mut)

In [ ]:
> length(unique(tcga.mut$Tumor_Sample_Barcode))

> head(table(tcga.mut$Tumor_Sample_Barcode))

> plot(table(tcga.mut$Tumor_Sample_Barcode))

> plot(sort(table(tcga.mut$Tumor_Sample_Barcode)))

> plot(log(sort(table(tcga.mut$Tumor_Sample_Barcode)), 2))

In [ ]:
install.packages("readxl")      # 최초 1회만
install.packages("tibble")   # 최초 1회만
library(readxl)
library(dplyr)
library(tibble)

raw_df <- read_excel("TCGA-CDR-SupplementalTableS1.xlsx", sheet = "TCGA-CDR")

tumortype_tcga <- raw_df %>%
  select(bcr_patient_barcode, type) %>%
  column_to_rownames("bcr_patient_barcode")


In [ ]:
# case number for tumor types
tumortype_tmb = table(tumortype_tcga$type)

# Tumor types with cases < 100 (UVM tumor type)
names(tumortype_tmb)[tumortype_tmb < 100]

# Code to illustrate TMB for 10 tumor types
par(mfrow=c(2,3))

for (tumortype in names(tumortype_tmb)[tumortype_tmb < 100]) {
  samples = rownames(tumortype_tcga)[tumortype_tcga$type == tumortype]
  samples_in_mut = tcga.mut$Tumor_Sample_Barcode[substr(tcga.mut$Tumor_Sample_Barcode, 1, 15) %in% paste0(samples, "-01")]

  plot(log(sort(table(samples_in_mut)), 2), type="p", ylab='', xaxt="n", xlab='', main=tumortype)
}

In [ ]:
table(tcga.mut$Hugo_Symbol)
tail(sort(table(tcga.mut$Hugo_Symbol)))

# UVM 샘플만 필터링 (예시: Tumor_Type 또는 bcr_patient_barcode 등으로 UVM 샘플 식별)
uvm_mut <- tcga.mut[tcga.mut$Tumor_Type == "UVM", ]  # Tumor_Type 컬럼이 있는 경우

# 또는 bcr_patient_barcode가 UVM 환자인 경우
# uvm_mut <- tcga.mut[tcga.mut$bcr_patient_barcode %in% uvm_patient_barcodes, ]

# UVM 샘플의 Hugo_Symbol별 mutation count 정렬
uvm_counts <- sort(table(uvm_mut$Hugo_Symbol))

# 결과 확인 (상위 6개)
tail(uvm_counts)



3강. Cancer Mutation 2

In [ ]:
install.packages("BiocManager")
BiocManager::install("GenVisR")

In [ ]:
# prepare tcga.mut & tumortype_tcga!

library(GenVisR)

UVM_sample = rownames(tumortype_tcga)[tumortype_tcga$type == "UVM"]
length(UVM_sample) # [1] 80

UVM_mut = tcga.mut[substr(tcga.mut$Tumor_Sample_Barcode, 1, 15) %in% paste0(UVM_sample, "-01"),]

colnames(UVM_mut)[12] = "amino.acid.change"

waterfall(UVM_mut, fileType="MAF", mainXlabel=TRUE, mainLabelCol="amino.acid.change", mainLabelSize=2)

# filter genes using MutSigCV

LUSC_mut_filtered = LUSC_mut[LUSC_mut$Hugo_Symbol %in% c("KEAP1", "PTEN", "TP53",
                                                         "CDKN2A", "MLL2", "NFE2L2", "RB1", "FBXW7", "NOTCH1", "PIK3CA"), ]

waterfall(LUSC_mut_filtered, fileType="MAF")

In [ ]:
# R script #
table(tcga.mut$Variant_Classification)

# Mutation types
par(mar=c(12, 6, 2, 2))
plot(table(tcga.mut$Variant_Classification), type='h', las=2)

In [ ]:
LUAD_sample = rownames(tumortype_tcga)[tumortype_tcga$type == "LUAD"]
LUAD_mut = tcga.mut[substr(tcga.mut$Tumor_Sample_Barcode, 1, 15) %in% paste0(LUAD_sample, "-01"),]

LUAD_mut_EGFR = LUAD_mut[LUAD_mut$Hugo_Symbol == "EGFR", ]

table(LUAD_mut_EGFR$HGVSp_Short)

In [ ]:
unique(LUAD_mut_EGFR[LUAD_mut_EGFR$HGVSp_Short == "p.L858R" |
    LUAD_mut_EGFR$HGVSp_Short == "p.E746_A750del", ]$Tumor_Sample_Barcode)

In [ ]:
# write LUAD-EGFR mutations into a file
write.table(LUAD_mut_EGFR[,c(2,3,4,7,8)], "LUAD_EGFR_mutations_forANNOVAR.txt",
sep="\t", quote=F, row.names=F, col.names=F)

In [ ]:
pik3ca_missense = tcga.mut[tcga.mut$Hugo_Symbol == "PIK3CA" &
    tcga.mut$Variant_Classification == "Missense_Mutation", ]

a = regexpr("[0-9]+", pik3ca_missense$HGVSp_Short, perl=TRUE)
b = regmatches(pik3ca_missense$HGVSp_Short, a)
hist(as.numeric(b), breaks=100)

In [ ]:
par(mfrow=c(6,1), mar=c(2,2,2,1))

genes = c("PIK3CA", "CTNNB1", "BRAF", "KRAS", "EGFR", "ERBB2")

for (gene in genes) {

    gene_missense = tcga.mut[tcga.mut$Hugo_Symbol == gene &
                             tcga.mut$Variant_Classification == "Missense_Mutation", ]

    a = regexpr("[0-9]+", gene_missense$HGVSp_Short, perl=TRUE)
    b = regmatches(gene_missense$HGVSp_Short, a)

    barplot(table(as.numeric(b)), col="red", main=gene)
}

4강. Copy number alteration 1

In [ ]:
tumortype_tcga =
  read.table(pipe("pbpaste"), sep="\t",
  header = TRUE, row.names=1)

In [ ]:
tcga.seg = read.table("broad.mit.edu_PANCAN_Genome_Wide_SNP_6_whitelisted.seg",
                      header=T, sep="\t")
head(tcga.seg)

In [ ]:
# Assign 'tumor type' and 'tumor(01)/normal(10/11)' and save

tcga.seg_sample = unique(tcga.seg$Sample)

tumortype_seg = tumortype_tcga[substr(tcga.seg_sample, 1, 12), ]

tumornormal_seg = ifelse(substr(tcga.seg_sample, 14, 15) == "01", "tumor", "others")

sampleAnnotation = cbind(tcga.seg_sample, tumortype_seg, tumornormal_seg)
colnames(sampleAnnotation) = c("sample", "tumortype", "tumornormal")

write.table(sampleAnnotation, "tcga_seg_sampleAnnotation.txt", sep="\t", quote=F, row.names=F)

5강. Copy number alteration 2

In [ ]:
# numDeriv packages should be installed prior
install.packages("numDeriv")  # need to locate a server

# install downloaded ABSOLUTE package
install.packages("ABSOLUTE_1.0.6.tar.gz", repos = NULL, type="source")

# Run ABSOLUTE on GBM segfiles
library(ABSOLUTE)

write.table(tcga.seg[tcga.seg$Sample == "TCGA-02-0006-01B-01D-0182-01", ],
            "GBMseg_exampleCase_TCGA-02-0006.seg", sep="\t", quote=F, row.names=F)

RunAbsolute("GBMseg_exampleCase_TCGA-02-0006.seg", 0, 0.02, 0.95, 10,
            "TCGA-GBM", "SNP_6.0", "TCGA-02-0006", "ABSOLUTE_RESULTS",
            2000, 0, 0, "total", verbose=TRUE)